In [1]:
import pymysql
import pandas as pd

db = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='12345678', 
                    db='financialservice', cursorclass=pymysql.cursors.DictCursor)

dwh = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='12345678', 
                    db='datawarehouse', cursorclass=pymysql.cursors.DictCursor)

cursorInstance = db.cursor()
dwh_cursor = dwh.cursor()

In [2]:
def run(command, *args):
    cursorInstance.execute(command, args)
    
def dwh_run(command, *args):
    dwh_cursor.execute(command, args)

def get():
    return cursorInstance.fetchall()

def dwh_get():
    return dwh_cursor.fetchall()

**Extract**

In [3]:
run("SELECT * FROM branch")
branch = get()

In [4]:
run("SELECT * FROM customer")
customer = get()

In [5]:
run("SELECT * FROM loan_category")
loan_category = get()

In [6]:
run("SELECT * FROM loan")
loan = get()

In [7]:
run("SELECT * FROM loan_transaction")
transaction = get()

**Transform**

**Load**

In [8]:
dwh.rollback()

print(len(branch))
for b in branch: 
    address = b['Address']
    print(b['BranchID'])
    print(address)
    number, street, city, province = address.split(',')
    print(number, street, city, province)
    sql = 'INSERT INTO branch_dimension VALUES (%s, %s, %s, %s)'
    dwh_run(sql, b['BranchID'], street, city, province)

dwh.commit()

50
0
25508, Đường Kim Ngọc, Thành phố Vĩnh Yên, Tỉnh Vĩnh Phúc
25508  Đường Kim Ngọc  Thành phố Vĩnh Yên  Tỉnh Vĩnh Phúc
1
049, Hai Bà Trưng, Quận Hoàn Kiếm, Thành phố Hà Nội
049  Hai Bà Trưng  Quận Hoàn Kiếm  Thành phố Hà Nội
10
2709, Võ Văn Kiệt, Quận 1, Thành phố Hồ Chí Minh
2709  Võ Văn Kiệt  Quận 1  Thành phố Hồ Chí Minh
11
809, Hàm Nghi, Quận 1, Thành phố Hồ Chí Minh
809  Hàm Nghi  Quận 1  Thành phố Hồ Chí Minh
12
853, Lê Thánh Tôn, Quận 1, Thành phố Hồ Chí Minh
853  Lê Thánh Tôn  Quận 1  Thành phố Hồ Chí Minh
13
9002, Bùi Thị Xuân, Quận 1, Thành phố Hồ Chí Minh
9002  Bùi Thị Xuân  Quận 1  Thành phố Hồ Chí Minh
14
990, Phó Đức Chính, Quận 1, Thành phố Hồ Chí Minh
990  Phó Đức Chính  Quận 1  Thành phố Hồ Chí Minh
15
5238, Nguyễn Công Trứ, Quận 1, Thành phố Hồ Chí Minh
5238  Nguyễn Công Trứ  Quận 1  Thành phố Hồ Chí Minh
16
704, Võ Văn Tần, Quận 3, Thành phố Hồ Chí Minh
704  Võ Văn Tần  Quận 3  Thành phố Hồ Chí Minh
17
258, Nam Kỳ Khởi Nghĩa, Quận 3, Thành phố Hồ Chí Minh
258  Nam 

In [9]:
dwh.rollback()

for lc in loan_category:
    sql = 'INSERT INTO loan_category_dimension VALUES (%s, %s, %s, %s, %s)'
    dwh_run(sql, lc['CategoryID'], lc['LifeSpan'], lc['Rate'], lc['Purpose'], lc['IsSecured'])
    
dwh.commit()

In [10]:
dwh.rollback()

from datetime import date, timedelta

sdate = date(2010, 1, 1)   # start date
edate = date(2030, 12, 31)   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    sql = 'INSERT INTO date_dimension VALUES (%s, %s, %s, %s, %s)'
    dwh_run(sql, i, day.day, day.month, day.year, (day.month - 1) // 3 + 1)
    
dwh.commit()

In [11]:
dwh.rollback()

from datetime import date, timedelta

sdate = date(2010, 1, 1)   # start date
edate = date(2030, 12, 31)   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    sql = 'INSERT INTO loan_started_date_dimension VALUES (%s, %s, %s, %s, %s)'
    dwh_run(sql, i, day.day, day.month, day.year, (day.month - 1) // 3 + 1)
    
dwh.commit()

In [12]:
dwh.rollback()

for b in customer: 
    sql = 'INSERT INTO customer_dimension VALUES (%s, %s, %s)'
    dwh_run(sql, b['ID'], b['Name'], b['DOB'])

dwh.commit()

In [13]:
from tqdm import tqdm

dwh.rollback()

for l in tqdm(loan): 
    key = l['LoanID']
    available_amount = l['AvailableAmount']
    date = l['StartDate']
    
    dwh_run(f"SELECT LoanStartedDateKey FROM loan_started_date_dimension WHERE LoanStartedDate = {date.day} AND LoanStartedMonth = {date.month} AND LoanStartedYear = {date.year}")
    date_id = dwh_get()[0]['LoanStartedDateKey']
    sql = 'INSERT INTO loan_dimension VALUES (%s, %s, %s)'
    dwh_run(sql, key, available_amount, date_id)

dwh.commit()

100%|██████████| 4023/4023 [00:10<00:00, 400.58it/s]


In [14]:
import datetime 
from datetime import timedelta

id = 0

dwh.rollback()

age_bands = [18, 25, 30, 35, 40, 45, 50, 55, 60]
income_bands = [0, 1000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 23000, 25000, 27000, 30000]

def get_age_band(a):
    for i in range(0, len(age_bands) - 1):
        if (a >= age_bands[i]) and (a < age_bands[i + 1]):
            return age_bands[i]
    
    return 60

def get_income_band(a):
    for i in range(0, len(income_bands) - 1):
        if (a >= income_bands[i]) and (a < income_bands[i + 1]):
            return income_bands[i]
    
    return 30000

for age_band in age_bands:
    for income_band in income_bands:
        for marital in range(4):
            for is_employed in range(2):
                id += 1
                sql = 'INSERT INTO customer_demographic_dimension VALUES (%s, %s, %s, %s, %s)'
                dwh_run(sql, id, age_band, income_band, marital, is_employed)

print(id)
    
dwh.commit()

1152


In [15]:
from tqdm import tqdm
import datetime

for t in tqdm(transaction):
    date = t['Date']
    dwh_run(f"SELECT DateKey FROM date_dimension WHERE Date = {date.day} AND Month = {date.month} AND Year = {date.year}")
    date_id = dwh_get()[0]['DateKey']
    
    run(f"SELECT AccountID, CategoryID FROM loan WHERE LoanID = {t['LoanID']}")
    result = get()[0]
    category_id = result['CategoryID']
    account_id = result['AccountID']
    
    run(f"SELECT CustomerID, BranchID FROM account WHERE AccountID = {account_id}")
    result = get()[0]
    customer_id = result['CustomerID']
    branch_id = result['BranchID']
    
    run(f"SELECT DOB, MaritalStatus, Income, Job FROM customer WHERE ID = {customer_id}")
    result = get()[0]
    dob = result['DOB']
    marital = result['MaritalStatus']
    income = result['Income']
    is_employed = result['Job']
    
    now = datetime.datetime.now()
    time_difference = now - dob
    age = time_difference.days // 365
    age_band = get_age_band(age)
    income_band = get_income_band(income_band)
        
    dwh_run(f"SELECT CustomerDemographicKey FROM customer_demographic_dimension WHERE AgeBand = {age_band} AND IncomeBand = {income_band} AND MaritalStatus = {marital} AND IsEmployed = {is_employed}")
    key = dwh_get()[0]['CustomerDemographicKey']
    loan_id = t['LoanID']
    
    sql = f'INSERT INTO loan_transaction_facts VALUES (%s, %s, %s, %s, %s, %s, %s)'
    dwh_run(sql, date_id, branch_id, category_id, customer_id, key, loan_id, t['Amount'])

dwh.commit()

100%|██████████| 60694/60694 [16:00<00:00, 63.18it/s]
